In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
import keras.ops as K

In [ ]:
def cbam_block(inputs, ratio=8):
    channel = inputs.shape[-1]

    avg_pool = layers.GlobalAveragePooling2D(keepdims=True)(inputs)
    max_pool = layers.GlobalMaxPooling2D(keepdims=True)(inputs)

    mlp = keras.Sequential([
        layers.Dense(channel // ratio, activation='relu'),
        layers.Dense(channel)
    ])

    avg_out = mlp(avg_pool)
    max_out = mlp(max_pool)

    channel_attention = layers.Activation("sigmoid")(avg_out + max_out)
    channel_refined = layers.Multiply()([inputs, channel_attention])
    avg_spatial = K.mean(channel_refined, axis=-1, keepdims=True)
    max_spatial = K.max(channel_refined, axis=-1, keepdims=True)

    concat = layers.Concatenate(axis=-1)([avg_spatial, max_spatial])

    spatial_attention = layers.Conv2D(
        filters=1,
        kernel_size=7,
        padding="same",
        activation="sigmoid"
    )(concat)

    refined = layers.Multiply()([channel_refined, spatial_attention])
    return refined

In [4]:
def build_model():
    inputs = keras.Input(shape=(224, 224, 3))

    base = keras.applications.EfficientNetB0(
        include_top=False,
        input_shape=(224, 224, 3),
        weights="imagenet"
    )

    x = keras.applications.efficientnet.preprocess_input(inputs)
    x = base(x)

    x = cbam_block(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(7, activation="softmax")(x)

    model = keras.Model(inputs, outputs)
    return model

model = build_model()
model.summary()

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb0      │ (None, 7, 7,      │  4,049,571 │ input_layer[0][0] │
│ (Functional)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1, 1,      │          0 │ efficientnetb0[0… │
│ (GlobalAveragePool… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 1, 1,      │          0 │ efficientnetb0[0… │
│ (GlobalMaxPooling2… │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 1, 1,      │    411,040 │ global_average_p… │
│ (Sequential)        │ 1280)             │            │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 1, 1,      │          0 │ sequential[0][0], │
│                     │ 1280)             │            │ sequential[1][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 1, 1,      │          0 │ add[0][0]         │
│ (Activation)        │ 1280)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 7, 7,      │          0 │ efficientnetb0[0… │
│                     │ 1280)             │            │ activation[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ mean (Mean)         │ (None, 7, 7, 1)   │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max (Max)           │ (None, 7, 7, 1)   │          0 │ multiply[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 7, 7, 2)   │          0 │ mean[0][0],       │
│ (Concatenate)       │                   │            │ max[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 7, 7, 1)   │         99 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_1          │ (None, 7, 7,      │          0 │ multiply[0][0],   │
│ (Multiply)          │ 1280)             │            │ conv2d[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1280)      │          0 │ multiply_1[0][0]  │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1280)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 7)         │      8,967 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 4,469,677 (17.05 MB)

 Trainable params: 4,427,654 (16.89 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [ ]:
import tensorflow as tf
from tensorflow import keras

IMG_SIZE = (224, 224)
BATCH_SIZE = 8
SEED = 42

def load_dataset(path):
    return keras.utils.image_dataset_from_directory(
        path,
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True,
        label_mode="categorical"
    )

paths = [
    "C://Users//manis//Downloads//FER-13//train",
    "C://Users//manis//Downloads//AffectNet//Train",
]

datasets = [load_dataset(p) for p in paths]

full_ds = datasets[0]
for ds in datasets[1:]:
    full_ds = full_ds.concatenate(ds)

# Shuffle the combined dataset
full_ds = full_ds.shuffle(2000, seed=SEED)

val_size = 0.2
total_batches = tf.data.experimental.cardinality(full_ds).numpy()
val_batches = int(total_batches * val_size)

val_ds = full_ds.take(val_batches)
train_ds = full_ds.skip(val_batches)

data_aug = keras.Sequential([
    keras.layers.RandomFlip("horizontal"),
    keras.layers.RandomRotation(0.1),
    keras.layers.RandomZoom(0.1)
])

train_ds = train_ds.map(lambda x, y: (data_aug(x, training=True), y),
                        num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(1)
val_ds = val_ds.prefetch(1)


Found 28709 files belonging to 7 classes.
Found 14549 files belonging to 7 classes.


In [6]:
model.compile(
    optimizer=keras.optimizers.Adam(1e-4),
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "best_cbam_model.h5",
        monitor="val_accuracy",
        save_best_only=True
    ),
    keras.callbacks.EarlyStopping(
        patience=8,
        monitor="val_loss",
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        factor=0.3,
        monitor="val_loss",
        patience=3,
        min_lr=1e-6
    )
]

In [7]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30
 449/4327 ━━━━━━━━━━━━━━━━━━━━ 26:19 407ms/step - accuracy: 0.2785 - loss: 1.7852

KeyboardInterrupt: 

In [7]:
model.save("cbam_efficientnetb0_v2_combinedDatasets.h5")